In [ ]:
# 20250816

In [ ]:
# https://csp.gitbook.io/langchain-for-beginners/ch13-langchain-expression-language-lcel/10.-runtime-arguments-binding

In [ ]:
# 10. Runtime Arguments Binding

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

In [4]:
# Load environment variables from .env file
load_dotenv()

True

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # Write the following equation using algebraic symbols, then solve it.
            "Write out the following equation using algebraic symbols then solve it. "
            "Use the format\n\nEQUATION:...\nSOLUTION:...\n\n",
        ),
        (
            "human",
            "{equation_statement}",  # Receives the equation sentence entered by the user as a variable.
        ),
    ]
)

In [10]:
# Initialize the ChatOpenAI model and set the temperature to 0.
model = ChatGroq( model="llama3-70b-8192", temperature=0)


In [11]:
# It takes an equation sentence as input and passes it to the prompt, and parses the results generated by the model into a string.
runnable = (
    {"equation_statement": RunnablePassthrough()} | prompt | model | StrOutputParser()
)

In [12]:
# Enter an example equation sentence and print the result.
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x³ + 7 = 12

SOLUTION:

To solve for x, subtract 7 from both sides of the equation:

x³ = 12 - 7
x³ = 5

Take the cube root of both sides:

x = ∛5
x = 1.71 (approximately)

So, the value of x is approximately 1.71.


In [13]:
runnable = (
    # Create an executable passthrough object and assign it to the "equation_statement" key.
    {"equation_statement": RunnablePassthrough()}
    | prompt  # Add a prompt to the pipeline.
    | model.bind(
        stop="SOLUTION"
    )  # Bind the model and set it to stop generating on the "SOLUTION" token.
    | StrOutputParser()  # Add a string output parser to the pipeline.
)
# "x raised to the third plus seven equals 12"Runs the pipeline with input called and outputs the results.
print(runnable.invoke("x raised to the third plus seven equals 12"))

EQUATION: x³ + 7 = 12




In [14]:
openai_function = {
    "name": "solver",  # Function name
    # Description of functions: Establish and solve equations.
    "description": "Formulates and solves an equation",
    "parameters": {  # Function parameters
        "type": "object",  # Parameter type: Object
        "properties": {  # Attributes of parameters
            "equation": {  # Equation properties
                "type": "string",  # Type of equation: string
                "description": "The algebraic expression of the equation",  # Algebraic representation of equations
            },
            "solution": {  # Answer property
                "type": "string",  # Type of answer: string
                "description": "The solution to the equation",  # 방정식의 해답
            },
        },
        "required": ["equation", "solution"],  # Required parameters: Equation and solution
    },
}

In [16]:
# Write the following equation using algebraic notation, then solve it.
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatGroq( model="llama3-70b-8192", temperature=0).bind(
    function_call={"name": "solver"},  # openai_function schema Binds to.
    functions=[openai_function],
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
# Adding 7 to the cube of x is equal to 12.
runnable.invoke("x raised to the third plus seven equals 12")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"equation":"x^3 + 7 = 12","solution":""}', 'name': 'solver'}}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 808, 'total_tokens': 852, 'completion_time': 0.112960138, 'prompt_time': 0.069657987, 'queue_time': 0.143169396, 'total_time': 0.182618125}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'finish_reason': 'function_call', 'logprobs': None}, id='run--05fac05d-3a3a-43ed-90a6-b99684313954-0', usage_metadata={'input_tokens': 808, 'output_tokens': 44, 'total_tokens': 852})

In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",  # The name of the function that gets the current weather.
            "description": "Get the current weather for a given location", # Description of the function
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City and state, e.g. San Francisco, CA", # Description of location parameter
                    },
                    # Temperature unit parameter (Celsius or Fahrenheit)
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],  # Specifying required parameters
            },
        },
    }
]

In [18]:
# Initialize the ChatOpenAI model and bind the tools.
model = ChatGroq( model="llama3-70b-8192", temperature=0).bind(tools=tools)
# Call the model and ask it about the weather in San Francisco, New York, and Los Angeles.
model.invoke("Tell me the current weather in San Francisco, New York, and Los Angeles?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'z4gq7vcjz', 'function': {'arguments': '{"location":"San Francisco, CA","unit":"fahrenheit"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'ww80qtqft', 'function': {'arguments': '{"location":"New York, NY","unit":"fahrenheit"}', 'name': 'get_current_weather'}, 'type': 'function'}, {'id': 'xrn3dwsry', 'function': {'arguments': '{"location":"Los Angeles, CA","unit":"fahrenheit"}', 'name': 'get_current_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 947, 'total_tokens': 1051, 'completion_time': 0.200381252, 'prompt_time': 0.077178936, 'queue_time': 0.143323298, 'total_time': 0.277560188}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_bf16903a67', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--3160fa2e-8721-4b4f-a509-c018b7576976-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': 'San Francisco, CA', 'unit'